In [2]:
from fastai import *
from fastai.basics import *
from fastai.vision.all import *

import numpy as np
import pandas as pd

from torch.utils.data import TensorDataset

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
path = Path('/kaggle/input/digit-recognizer')

## Loading Training Data

In [4]:
train_df = pd.read_csv(path/'train.csv');train_df

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41996,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41997,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41998,6,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
y_data = tensor(train_df['label']);y_data.shape

torch.Size([42000])

In [6]:
x_data = tensor(train_df.drop('label',axis=1).values).reshape(-1,28,28).unsqueeze(1);x_data.shape

torch.Size([42000, 1, 28, 28])

In [8]:
# convert to 3 channel image tensors
x_data = torch.cat([torch.cat([x_data[i],x_data[i],x_data[i],])[None] for i in range(x_data.shape[0])]).float();x_data.shape

torch.Size([42000, 3, 28, 28])

In [10]:
# Somewhat manually do a validation split; can from_dsets do this for me somehow?
n = x_data.shape[0]
idxs = torch.randperm(n)
cut = int(0.8*n)
x_train = x_data[idxs[:cut]]
y_train = y_data[idxs[:cut]]
x_valid = x_data[idxs[cut:]]
y_valid = y_data[idxs[cut:]]

In [11]:
x_train.shape,y_train.shape,x_valid.shape,y_valid.shape

(torch.Size([33600, 3, 28, 28]),
 torch.Size([33600]),
 torch.Size([8400, 3, 28, 28]),
 torch.Size([8400]))

In [12]:
train_data = TensorDataset(x_train,y_train)
valid_data = TensorDataset(x_valid,y_valid)

batch_tfms = [*aug_transforms(do_flip=False,), Normalize.from_stats(*imagenet_stats)]

dls = ImageDataLoaders.from_dsets(train_data,valid_data,batch_tfms=[batch_tfms],bs=128)

## Let's Train

In [13]:

# dsets example https://docs.fast.ai/tutorial.siamese.html
learn = vision_learner(dls,resnet34,metrics=accuracy,loss_func=CrossEntropyLossFlat(),n_out=10)

Downloading: "https://download.pytorch.org/models/resnet34-b627a593.pth" to /root/.cache/torch/hub/checkpoints/resnet34-b627a593.pth


  0%|          | 0.00/83.3M [00:00<?, ?B/s]

In [ ]:
# calculate a sane learning rate
lrs = learn.lr_find()

In [ ]:
lrs

In [ ]:
# Some callbacks to manage training
cbs = [
#GradientAccumulation(),
#    MixedPrecision(),
    SaveModelCallback(monitor='accuracy', comp=np.greater, min_delta=0.001),
    ReduceLROnPlateau(monitor='accuracy', comp=np.greater, min_delta=0.001, patience=2),
#MixUp(0.4),
#EarlyStoppingCallback(monitor='accuracy', comp=np.greater, min_delta=0.001, patience=3),
      ]

In [ ]:
learn.fine_tune(15, lrs.valley,cbs=cbs)

## Review Submission Sample

In [ ]:
sample_submission = pd.read_csv(path/'sample_submission.csv',);sample_submission

## Generate Submission File

In [ ]:
test_df = pd.read_csv(path/'test.csv');test_df

In [ ]:
# pixel data from dataframe into a tensor
td = tensor(test_df.values).float().reshape(-1,28,28).unsqueeze(1);td.shape

In [ ]:
# convert to 3 channel rgb
td = torch.cat([torch.cat([td[i],td[i],td[i]]).unsqueeze(0) for i in range(td.shape[0])]);td.shape

In [ ]:
# # evaluate model on whole test dataset; I had trouble sort out how to get this to work... so used learn.predict instead... 
# test_dl = learn.dls.test_dl([[td[i] for i in range(td.shape[0])]])
# preds, _, decoded = learn.get_preds(dl=test_dl, with_decoded=True)

In [ ]:
def predict_quietly(item):
    with learn.no_bar(), learn.no_logging():
        return learn.predict(item)

In [ ]:
from tqdm import tqdm

In [ ]:
# takes a bit of time due to doing it one at a time... need to debug get_preds above in this scenario
decoded = [predict_quietly([td[i]])[0].item() for i in tqdm(range(td.shape[0]))]

In [ ]:
# create df and generate csv from decoded predictions
submission = pd.DataFrame(np.arange(len(td))+1, columns=['ImageId'])
submission['Label'] = decoded
submission.head()

submission.to_csv("submission.csv",index=False)